<a href="https://colab.research.google.com/github/SUPERREALCODER/buisness_report_rag/blob/main/buisness_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install llmware

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 13.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidi

In [2]:
  from llmware.setup import Setup
  sample_files_path = Setup().load_sample_files()


In [6]:



import os
import re
from llmware.prompts import Prompt, HumanInTheLoop
from llmware.setup import Setup
from llmware.configs import LLMWareConfig
from llmware.retrieval import Query
from llmware.library import Library


def busines_report_analysis_from_library (model_name, verbose=False):




    print (f"\n > Loading the llmware sample files...")
    sample_files_path = Setup().load_sample_files()
    contracts_path = os.path.join(sample_files_path,"FinDocs")

    contracts_lib = Library().create_new_library("example4_library")
    contracts_lib.add_files(contracts_path)

    # questions that we want to ask each report
    question_list = [
    {"topic": "Company Overview", "llm_query": "What is the mission statement of the company?"},
    {"topic": "Financial Highlights", "llm_query": "What are the key financial highlights for the year?"},
    {"topic": "Revenue", "llm_query": "What is the total revenue reported for the year?"},
    {"topic": "Net Income", "llm_query": "What is the net income for the year?"},
    {"topic": "Cash Flow", "llm_query": "What are the net cash flows from operating activities?"},
    {"topic": "Auditor's Report", "llm_query": "What is the auditor's opinion on the financial statements?"},
    {"topic": "Risk Factors", "llm_query": "What are the primary risk factors mentioned?"},
    {"topic": "Market Position", "llm_query": "How does the company describe its market position?"},
    {"topic": "Board of Directors", "llm_query": "Who are the members of the board of directors?"},
    {"topic": "Executive Compensation", "llm_query": "What is the total compensation for the CEO?"},
    {"topic": "Strategic Initiatives", "llm_query": "What are the key strategic initiatives outlined for the upcoming year?"},
    {"topic": "Dividends", "llm_query": "What is the company's dividend policy?"},
    {"topic": "Environmental Initiatives", "llm_query": "What environmental initiatives did the company undertake?"},
    {"topic": "Corporate Social Responsibility", "llm_query": "What CSR activities are highlighted in the report?"},
    {"topic": "Shareholder Information", "llm_query": "How did the company's stock perform over the past year?"},
    {"topic": "Business Segments", "llm_query": "What are the different business segments and their revenues?"},
    {"topic": "Governance", "llm_query": "What governance practices are described in the report?"},
    {"topic": "Future Outlook", "llm_query": "What is the management's outlook for the next fiscal year?"},
    {"topic": "Legal Proceedings", "llm_query": "Are there any significant legal proceedings mentioned?"},
    {"topic": "Research and Development", "llm_query": "How much did the company spend on research and development?"}
]
    print (f"\n > Loading model {model_name}...")

    q = Query(contracts_lib)

    # get a list of all of the unique documents in the library

    # doc id list
    doc_list = q.list_doc_id()
    print("update: document id list - ", doc_list)

    # filename list
    fn_list = q.list_doc_fn()
    print("update: filename list - ", fn_list)

    prompter = Prompt().load_model(model_name)

    for i, doc_id in enumerate(doc_list):

        print("\nAnalyzing contract: ", str(i+1), doc_id, fn_list[i])

        print("LLM Responses:")

        for question in question_list:

            query_topic = question["topic"]
            llm_question = question["llm_query"]

            doc_filter = {"doc_ID": [doc_id]}
            query_results = q.text_query_with_document_filter(query_topic,doc_filter,result_count=5,exact_mode=True)

            if verbose:
                # this will display the query results from the query above
                for j, qr in enumerate(query_results):
                    print("update: querying document - ", query_topic, j, doc_filter, qr)

            source = prompter.add_source_query_results(query_results)

            #   *** this is the call to the llm with the source packaged in the context automatically ***
            responses = prompter.prompt_with_source(llm_question, prompt_name="default_with_context", temperature=0.3)

            #   unpacking the results from the LLM
            for r, response in enumerate(responses):
                print("update: llm response -  ", llm_question, re.sub("[\n]"," ", response["llm_response"]).strip())

            # We're done with this contract, clear the source from the prompt
            prompter.clear_source_materials()

    #   Save jsonl report to jsonl to /prompt_history folder
    print("\nPrompt state saved at: ", os.path.join(LLMWareConfig.get_prompt_path(),prompter.prompt_id))
    prompter.save_state()

    #   Save csv report that includes the model, response, prompt, and evidence for human-in-the-loop review
    csv_output = HumanInTheLoop(prompter).export_current_interaction_to_csv()
    print("\nCSV output saved at:  ", csv_output)

    return 0







if __name__ == "__main__":


    LLMWareConfig().set_active_db("sqlite")

    example_models = ["llmware/bling-1b-0.1", "llmware/bling-tiny-llama-v0", "llmware/dragon-yi-6b-gguf"]




    model_name = example_models[0]


    busines_report_analysis_from_library(model_name)




 > Loading the llmware sample files...

 > Loading model llmware/bling-1b-0.1...
update: document id list -  [2, 3, 4, 5, 6, 7]
update: filename list -  ['accenture-fiscal-2021-annual-report.pdf', 'q3-fy22-earnings-release.pdf', 'Q2-2022-Amazon-Earnings-Release.pdf', 'Amazon-2021-Annual-Report.pdf', 'Annual_Report_2003.pdf', 'IBM 2021 Annual report.pdf']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.11G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

 •Net cash flows from operating activities, as a percentage of net cash flows from investing activities.
update: llm response -   What is the auditor's opinion on the financial statements? Not Found.
update: llm response -   What are the primary risk factors mentioned? 1.  Qualitative factors beyond the quantitative financial metrics are a key consideration in the determination of individual executive compensation.
update: llm response -   How does the company describe its market position? The company is currently the market leader at twice the size of its nearest competitor.
update: llm response -   Who are the members of the board of directors? 12 directors.
update: llm response -   What is the total compensation for the CEO? Not Found.
 •Our long-standing values have served IBM for decades. As the world continues to battle a global public health crisis, we are committed to building upon our long-standing values. As the world continues to battle a global public health crisis, we are 

 •Executive session led by independent Lead Director


update: llm response -   How did the company's stock perform over the past year? The stock price has increased by $1.00 per share.
 23.


 •The Board has continued to review the independence of non-management directors, and the Board has determined that independent directors are led by the Chair of the Board and the Chair of the Directors and Corporate Governance, and the Chair of the Audit, Directors and Corporate Governance, and the Chair of the Executive Compensation and Management Resources Committees.


update: llm response -   What is the management's outlook for the next fiscal year? Not Found.
update: llm response -   Are there any significant legal proceedings mentioned? Yes, there are two significant legal proceedings.
update: llm response -   How much did the company spend on research and development? Not Found.

Prompt state saved at:  /root/llmware_data/prompt_history/fc3a3845-f562-406b-bc4e-869c1a15e5ba

CSV output saved at:   {'report_name': 'interaction_report_Tue May 21 11:19:22 2024.csv', 'report_fp': '/root/llmware_data/prompt_history/interaction_report_Tue May 21 11:19:22 2024.csv', 'results': 59}
